In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vo

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Wed Aug 18 17:55:29 PDT 2021


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
project_results_dir = '/ihme/costeffectiveness/results/vivarium_ciff_sam'
model_name = 'v2.5_stunting'
model_timestamp = '2021_08_05_16_17_12'
username = 'ndbs'

In [5]:
model_count_data_dir = f'{project_results_dir}/{model_name}/ciff_sam/{model_timestamp}/count_data/'

project_vv_directory_name = 'ciff_malnutrition/verification_and_validation'

output_dir = f'/ihme/homes/{username}/vivarium_results/{project_vv_directory_name}/{model_name}'
share_output_dir = f'/share/scratch/users/ndbs/vivarium_results/{project_vv_directory_name}/{model_name}'
j_output_dir = f'/home/j/Project/simulation_science/{project_vv_directory_name}/{model_name}'

# Create the output directories if they don't exist
# Note from Path.mkdir() documentation:
#   "If mode is given, it is combined with the process’ umask value to determine the file mode and access flags."
#
# I don't know what this notebook process' umask value will be, so I don't know if this will actually result
# in the correct (most permissive) permissions for the directories...
# for directory in [output_dir, share_output_dir, j_output_dir]:
#     Path(directory).mkdir(mode=0o777, parents=True, exist_ok=True)

# Load transformed count data

In [6]:
data = vto.VivariumTransformedOutput.from_directory(model_count_data_dir)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [7]:
data.wasting_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [8]:
data.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,1245.0,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,3803.0,late_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,29178.0,1-5_months
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,34038.0,6-11_months
...,...,...,...,...,...,...,...
5036,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,3330.0,1-5_months
5037,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,4391.0,6-11_months
5038,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,8691.0,12_to_23_months
5039,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,23097.0,2_to_4


In [9]:
data.disease_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,state_person_time,29,baseline,43.489391,cat4,early_neonatal
1,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat3,early_neonatal
2,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat2,early_neonatal
3,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,susceptible_to_measles,state_person_time,946,baseline,121118.023272,cat4,2_to_4
17277,male,2026,susceptible_to_measles,state_person_time,946,baseline,67325.987680,cat3,2_to_4
17278,male,2026,susceptible_to_measles,state_person_time,946,baseline,66608.265572,cat2,2_to_4
17279,male,2026,susceptible_to_measles,state_person_time,946,baseline,63010.346338,cat1,2_to_4


In [10]:
data.wasting_state_person_time.cause.unique()

array(['mild_child_wasting', 'moderate_acute_malnutrition',
       'severe_acute_malnutrition', 'susceptible_to_child_wasting'],
      dtype=object)

In [14]:
data.wasting_state_person_time.query("cause in []")

,sex,year,cause,measure,input_draw,scenario,value,age


In [16]:
# Transition counts divided by total person time (not really a useful measure...)
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast=['measure']
)

,year,age,sex,input_draw,scenario,measure,value
0,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,0.661024
1,2022,1-5_months,female,29,baseline,mild_child_wasting_to_susceptible_to_child_was...,0.074263
2,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition_to_mild_child_wast...,0.578107
3,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,0.145444
...,...,...,...,...,...,...,...
5036,2026,late_neonatal,male,946,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,0.232412
5037,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition_to_mild_child_wastin...,0.128548
5038,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition_to_moderate_acute_ma...,0.110276
5039,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,0.063473


# Test ratio function refactored to call `stratify`

Same result as above -- good

In [21]:
# Transition counts divided by total person time (not really a useful measure...)
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast=['measure']
)

,year,age,sex,input_draw,scenario,measure,value
0,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,0.661024
1,2022,1-5_months,female,29,baseline,mild_child_wasting_to_susceptible_to_child_was...,0.074263
2,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition_to_mild_child_wast...,0.578107
3,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,0.145444
...,...,...,...,...,...,...,...
5036,2026,late_neonatal,male,946,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,0.232412
5037,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition_to_mild_child_wastin...,0.128548
5038,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition_to_moderate_acute_ma...,0.110276
5039,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,0.063473


# Add `denominator_broadcast` parameter so I can divide transition counts by person-time in each wasting state (mislabeled `'cause'`)

In [25]:
tcount_to_pt = vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast=['measure'],
    denominator_broadcast=['cause'],
)
tcount_to_pt

,year,age,sex,input_draw,scenario,measure,cause,value
0,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,mild_child_wasting,3.268528
1,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,moderate_acute_malnutrition,8.254240
2,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,severe_acute_malnutrition,31.588639
3,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,susceptible_to_child_wasting,0.948723
...,...,...,...,...,...,...,...,...
20156,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.306930
20157,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,moderate_acute_malnutrition,0.673521
20158,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,severe_acute_malnutrition,1.797606
20159,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,0.095641


# Compute SAM treatment rate according to sim

In [26]:
tcount_to_pt.measure.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
       'mild_child_wasting_to_susceptible_to_child_wasting_event_count',
       'moderate_acute_malnutrition_to_mild_child_wasting_event_count',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
       'severe_acute_malnutrition_to_mild_child_wasting_event_count',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count',
       'susceptible_to_child_wasting_to_mild_child_wasting_event_count'],
      dtype=object)

In [28]:
tx_rate = tcount_to_pt.query(
    "measure == 'severe_acute_malnutrition_to_mild_child_wasting_event_count'"
    " and cause == 'severe_acute_malnutrition'"
)
tx_rate

,year,age,sex,input_draw,scenario,measure,cause,value
18,2022,1-5_months,female,29,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.763181
46,2022,1-5_months,female,223,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.673883
74,2022,1-5_months,female,232,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.672059
102,2022,1-5_months,female,357,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.800361
...,...,...,...,...,...,...,...,...
20066,2026,late_neonatal,male,650,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.909828
20094,2026,late_neonatal,male,680,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.765661
20122,2026,late_neonatal,male,829,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.749213
20150,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition,3.640607


In [31]:
tx_rate.value.describe(percentiles=[.025,.975])

count    720.000000
mean       3.659547
std        0.153683
min        2.803577
2.5%       3.180962
50%        3.693762
97.5%      3.865451
max        4.073268
Name: value, dtype: float64

In [32]:
.488*.75

0.366

In [53]:
(tx_rate.value * 54.5/365).describe(percentiles=[.025,.975])

count    720.000000
mean       0.546425
std        0.022947
min        0.418616
2.5%       0.474966
50%        0.551534
97.5%      0.577170
max        0.608200
Name: value, dtype: float64

# Compute SAM duration according to sim

In [36]:
df = data.wasting_transition_count
df.loc[df.measure.str.contains('to_severe_acute_malnutrition'), 'measure'].unique()

array(['moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count'],
      dtype=object)

In [37]:
sam_inv_duration = tcount_to_pt.query(
    "measure == 'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count'"
    " and cause == 'severe_acute_malnutrition'"
)
sam_inv_duration

,year,age,sex,input_draw,scenario,measure,cause,value
14,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.950410
42,2022,1-5_months,female,223,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.821591
70,2022,1-5_months,female,232,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.822543
98,2022,1-5_months,female,357,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,7.383500
...,...,...,...,...,...,...,...,...
20062,2026,late_neonatal,male,650,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,5.799380
20090,2026,late_neonatal,male,680,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.664285
20118,2026,late_neonatal,male,829,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.267150
20146,2026,late_neonatal,male,946,baseline,moderate_acute_malnutrition_to_severe_acute_ma...,severe_acute_malnutrition,6.582144


In [43]:
sam_inv_duration.value.describe(percentiles=[0.025, 0.975])

count    720.000000
mean       6.547674
std        0.333507
min        5.293646
2.5%       5.770885
50%        6.583668
97.5%      7.159918
max        7.548558
Name: value, dtype: float64

In [42]:
(1/sam_inv_duration.value).describe(percentiles=[0.025, 0.975])

count    720.000000
mean       0.153137
std        0.008121
min        0.132476
2.5%       0.139667
50%        0.151891
97.5%      0.173284
max        0.188906
Name: value, dtype: float64

In [54]:
0.132476 * 365

48.35374

In [44]:
0.153137 * 365

55.895005

In [55]:
0.188906 * 365

68.95069

In [45]:
.488/0.153137

3.186689043144374

In [68]:
index = sam_inv_duration.columns.difference(['value']).to_list()
sam_duration = 1 / sam_inv_duration.set_index(index)

365 * sam_duration.groupby(['age', 'cause', 'measure', 'scenario'])[['value']].describe()

value  \
                                                                                                         count   
age             cause                     measure                                            scenario            
1-5_months      severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   
12_to_23_months severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   
2_to_4          severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   
6-11_months     severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   
early_neonatal  severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   
late_neonatal   severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  43800.0   

                                                                                                                  \
                                                                                                            mean   
age             cause                     measure                                            scenario              
1-5_months      severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  53.627677   
12_to_23_months severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  56.682950   
2_to_4          severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  55.350144   
6-11_months     severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  54.410130   
early_neonatal  severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  58.659734   
late_neonatal   severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  56.640147   

                                                                                                                 \
                                                                                                            std   
age             cause                     measure                                            scenario             
1-5_months      severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  1.669645   
12_to_23_months severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  1.494795   
2_to_4          severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  0.569866   
6-11_months     severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  1.671975   
early_neonatal  severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  3.077855   
late_neonatal   severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  4.343725   

                                                                                                                  \
                                                                                                             min   
age             cause                     measure                                            scenario              
1-5_months      severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  49.292300   
12_to_23_months severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  53.569015   
2_to_4          severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  54.132872   
6-11_months     severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  50.288982   
early_neonatal  severe_acute_malnutrition moderate_acute_malnutrition_to_severe_acute_mal... baseline  52.537502   
late_neonatal   severe_acute_malnutrition moderate_acute_malnutrition

In [70]:
sam_duration

value
age           cause                     input_draw measure                                            scenario sex    year          
1-5_months    severe_acute_malnutrition 29         moderate_acute_malnutrition_to_severe_acute_mal... baseline female 2022  0.143876
                                        223        moderate_acute_malnutrition_to_severe_acute_mal... baseline female 2022  0.146593
                                        232        moderate_acute_malnutrition_to_severe_acute_mal... baseline female 2022  0.146573
                                        357        moderate_acute_malnutrition_to_severe_acute_mal... baseline female 2022  0.135437
...                                                                                                                              ...
late_neonatal severe_acute_malnutrition 650        moderate_acute_malnutrition_to_severe_acute_mal... baseline male   2026  0.172432
                                        680        moderate_acute_malnutrition_to_severe_acute_mal... baseline male   2026  0.150054
                                        829        moderate_acute_malnutrition_to_severe_acute_mal... baseline male   2026  0.159562
                                        946        moderate_acute_malnutrition_to_severe_acute_mal... baseline male   2026  0.151926

[720 rows x 1 columns]

In [76]:
tx_rate.set_index(index)

value
age           cause                     input_draw measure                                            scenario sex    year          
1-5_months    severe_acute_malnutrition 29         severe_acute_malnutrition_to_mild_child_wasting... baseline female 2022  3.763181
                                        223        severe_acute_malnutrition_to_mild_child_wasting... baseline female 2022  3.673883
                                        232        severe_acute_malnutrition_to_mild_child_wasting... baseline female 2022  3.672059
                                        357        severe_acute_malnutrition_to_mild_child_wasting... baseline female 2022  3.800361
...                                                                                                                              ...
late_neonatal severe_acute_malnutrition 650        severe_acute_malnutrition_to_mild_child_wasting... baseline male   2026  3.909828
                                        680        severe_acute_malnutrition_to_mild_child_wasting... baseline male   2026  3.765661
                                        829        severe_acute_malnutrition_to_mild_child_wasting... baseline male   2026  3.749213
                                        946        severe_acute_malnutrition_to_mild_child_wasting... baseline male   2026  3.640607

[720 rows x 1 columns]

## This is how Rajan says we calculated the `t1` arrow:

`t1 = sam_tx_coverage * (1/time_to_sam_tx_recovery)`

The calculated value 3.20 is fairly close to the tx rate 3.66 that we got above.

In [47]:
(0.488 * sam_inv_duration.value).describe(percentiles=[0.025, 0.975])

count    720.000000
mean       3.195265
std        0.162751
min        2.583299
2.5%       2.816192
50%        3.212830
97.5%      3.494040
max        3.683696
Name: value, dtype: float64

# Test `reset_index` parameter

In [50]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast=['measure'],
    denominator_broadcast=['cause'],
    reset_index=True
)

,year,age,sex,input_draw,scenario,measure,cause,value
0,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,mild_child_wasting,3.268528
1,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,moderate_acute_malnutrition,8.254240
2,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,severe_acute_malnutrition,31.588639
3,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,susceptible_to_child_wasting,0.948723
...,...,...,...,...,...,...,...,...
20156,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.306930
20157,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,moderate_acute_malnutrition,0.673521
20158,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,severe_acute_malnutrition,1.797606
20159,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,0.095641


In [51]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast=['measure'],
    denominator_broadcast=['cause'],
    reset_index=False
)

value
year age           sex    input_draw scenario measure                                            cause                                  
2022 1-5_months    female 29         baseline mild_child_wasting_to_moderate_acute_malnutriti... mild_child_wasting             3.268528
                                                                                                 moderate_acute_malnutrition    8.254240
                                                                                                 severe_acute_malnutrition     31.588639
                                                                                                 susceptible_to_child_wasting   0.948723
...                                                                                                                                  ...
2026 late_neonatal male   946        baseline susceptible_to_child_wasting_to_mild_child_wast... mild_child_wasting             0.306930
                                                                                                 moderate_acute_malnutrition    0.673521
                                                                                                 severe_acute_malnutrition      1.797606
                                                                                                 susceptible_to_child_wasting   0.095641

[20160 rows x 1 columns]

# Try computing percent treated

In [78]:
data.wasting_transition_count.measure.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
       'mild_child_wasting_to_susceptible_to_child_wasting_event_count',
       'moderate_acute_malnutrition_to_mild_child_wasting_event_count',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
       'severe_acute_malnutrition_to_mild_child_wasting_event_count',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count',
       'susceptible_to_child_wasting_to_mild_child_wasting_event_count'],
      dtype=object)

In [91]:
r2ux_count = data.wasting_transition_count.query("measure == 'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count'")

In [79]:
t1_count = data.wasting_transition_count.query("measure == 'severe_acute_malnutrition_to_mild_child_wasting_event_count'")
entering_sam = data.wasting_transition_count.query("measure == 'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count'")
entering_sam

,sex,year,measure,input_draw,scenario,value,age
216,female,2022,moderate_acute_malnutrition_to_severe_acute_ma...,29,baseline,274.0,early_neonatal
217,female,2022,moderate_acute_malnutrition_to_severe_acute_ma...,29,baseline,736.0,late_neonatal
218,female,2022,moderate_acute_malnutrition_to_severe_acute_ma...,29,baseline,6420.0,1-5_months
219,female,2022,moderate_acute_malnutrition_to_severe_acute_ma...,29,baseline,7454.0,6-11_months
...,...,...,...,...,...,...,...
4820,male,2026,moderate_acute_malnutrition_to_severe_acute_ma...,946,baseline,12719.0,1-5_months
4821,male,2026,moderate_acute_malnutrition_to_severe_acute_ma...,946,baseline,13308.0,6-11_months
4822,male,2026,moderate_acute_malnutrition_to_severe_acute_ma...,946,baseline,19839.0,12_to_23_months
4823,male,2026,moderate_acute_malnutrition_to_severe_acute_ma...,946,baseline,36682.0,2_to_4


In [82]:
t1_count

,sex,year,measure,input_draw,scenario,value,age
288,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,29,baseline,175.0,early_neonatal
289,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,29,baseline,448.0,late_neonatal
290,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,29,baseline,3476.0,1-5_months
291,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,29,baseline,4025.0,6-11_months
...,...,...,...,...,...,...,...
4892,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,946,baseline,6576.0,1-5_months
4893,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,946,baseline,7515.0,6-11_months
4894,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,946,baseline,11305.0,12_to_23_months
4895,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,946,baseline,20529.0,2_to_4


In [87]:
vo.ratio(
    t1_count,
    entering_sam,
    strata = ['age'],
    reset_index=False
).groupby(['age', 'scenario']).describe()

value                                          \
                         count      mean       std       min       25%   
age             scenario                                                 
1-5_months      baseline  12.0  0.542111  0.012163  0.518107  0.536216   
12_to_23_months baseline  12.0  0.573794  0.010299  0.552000  0.570574   
2_to_4          baseline  12.0  0.561220  0.002697  0.557370  0.559766   
6-11_months     baseline  12.0  0.552866  0.015739  0.521174  0.547466   
early_neonatal  baseline  12.0  0.552269  0.015941  0.527227  0.539739   
late_neonatal   baseline  12.0  0.573607  0.030523  0.527775  0.548423   

                                                        
                               50%       75%       max  
age             scenario                                
1-5_months      baseline  0.542767  0.549865  0.560162  
12_to_23_months baseline  0.572392  0.578521  0.596155  
2_to_4          baseline  0.560553  0.562595  0.566568  
6-11_months     baseline  0.552178  0.559527  0.586368  
early_neonatal  baseline  0.552313  0.560500  0.582553  
late_neonatal   baseline  0.573402  0.587618  0.642613

In [81]:
data.deaths.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'other_causes',
       'severe_acute_malnutrition'], dtype=object)

In [88]:
3.66/3.2

1.14375

In [89]:
.56/.488

1.1475409836065575

In [145]:
index_cols = t1_count.columns.difference(['value', 'measure']).to_list()

df = vo.ratio(
    t1_count,
    t1_count.set_index(index_cols) + r2ux_count.set_index(index_cols),
    strata='age',
)
df

,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,29,baseline,0.548127,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
1,1-5_months,223,baseline,0.542031,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
2,1-5_months,232,baseline,0.541958,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
3,1-5_months,357,baseline,0.545019,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
...,...,...,...,...,...,...,...
68,late_neonatal,650,baseline,0.544402,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
69,late_neonatal,680,baseline,0.546673,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
70,late_neonatal,829,baseline,0.545952,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1
71,late_neonatal,946,baseline,0.539960,severe_acute_malnutrition_to_mild_child_wastin...,severe_acute_malnutrition_to_mild_child_wastin...,1


In [146]:
describe(df)

,,,,,count,mean,std,min,2.5%,50%,97.5%,max
age,denominator_measure,multiplier,numerator_measure,scenario,,,,,,,,
1-5_months,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.543811,0.001668,0.541958,0.541978,0.543544,0.547273,0.548127
12_to_23_months,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.544796,0.001233,0.543103,0.543167,0.544659,0.546836,0.547053
2_to_4,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.544473,0.001220,0.542535,0.542679,0.544525,0.546475,0.546652
6-11_months,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.544237,0.001549,0.542406,0.542483,0.543890,0.547006,0.547547
early_neonatal,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.542323,0.007962,0.532609,0.532771,0.540823,0.557504,0.559659
late_neonatal,severe_acute_malnutrition_to_mild_child_wasting_event_countsevere_acute_malnutrition_to_moderate_acute_malnutrition_event_count,1,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,12.0,0.544477,0.003565,0.536520,0.537466,0.545317,0.549064,0.549538


In [125]:
t1_count.set_index(index_cols)

,,,,,measure,value
age,input_draw,scenario,sex,year,,
early_neonatal,29,baseline,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,175.0
late_neonatal,29,baseline,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,448.0
1-5_months,29,baseline,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,3476.0
6-11_months,29,baseline,female,2022,severe_acute_malnutrition_to_mild_child_wastin...,4025.0
...,...,...,...,...,...,...
1-5_months,946,baseline,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,6576.0
6-11_months,946,baseline,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,7515.0
12_to_23_months,946,baseline,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,11305.0
2_to_4,946,baseline,male,2026,severe_acute_malnutrition_to_mild_child_wastin...,20529.0


In [127]:
vo.marginalize(t1_count, [], reset_index=False)

value
age           input_draw measure                                            scenario sex    year        
1-5_months    29         severe_acute_malnutrition_to_mild_child_wasting... baseline female 2022  3476.0
                                                                                            2023  3362.0
                                                                                            2024  3413.0
                                                                                            2025  3424.0
...                                                                                                  ...
late_neonatal 946        severe_acute_malnutrition_to_mild_child_wasting... baseline male   2023   833.0
                                                                                            2024   811.0
                                                                                            2025   769.0
                                                                                            2026   802.0

[720 rows x 1 columns]

In [128]:
vo.stratify(t1_count, ['age', 'sex', 'year'], reset_index=False)

value
age           sex    year input_draw scenario        
1-5_months    female 2022 29         baseline  3476.0
                          223        baseline  3931.0
                          232        baseline  3492.0
                          357        baseline  3674.0
...                                               ...
late_neonatal male   2026 650        baseline   987.0
                          680        baseline   886.0
                          829        baseline   880.0
                          946        baseline   802.0

[720 rows x 1 columns]

In [98]:
t1_count.set_index(index_cols)

,,,,,,value
age,input_draw,measure,scenario,sex,year,
early_neonatal,29,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,female,2022,175.0
late_neonatal,29,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,female,2022,448.0
1-5_months,29,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,female,2022,3476.0
6-11_months,29,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,female,2022,4025.0
...,...,...,...,...,...,...
1-5_months,946,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,male,2026,6576.0
6-11_months,946,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,male,2026,7515.0
12_to_23_months,946,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,male,2026,11305.0
2_to_4,946,severe_acute_malnutrition_to_mild_child_wasting_event_count,baseline,male,2026,20529.0


In [97]:
r2ux_count.set_index(index_cols)

,,,,,,value
age,input_draw,measure,scenario,sex,year,
early_neonatal,29,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,female,2022,142.0
late_neonatal,29,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,female,2022,371.0
1-5_months,29,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,female,2022,2881.0
6-11_months,29,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,female,2022,3366.0
...,...,...,...,...,...,...
1-5_months,946,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,male,2026,5523.0
6-11_months,946,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,male,2026,6243.0
12_to_23_months,946,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,male,2026,9510.0
2_to_4,946,severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count,baseline,male,2026,17241.0


# Try recording what ratio function does

In [100]:
9*8

72

In [103]:
data.wasting_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [105]:
vo.stratify(data.wasting_state_person_time, ['age', 'sex', 'year'])

,age,sex,year,input_draw,scenario,value
0,1-5_months,female,2022,29,baseline,44140.585900
1,1-5_months,female,2022,223,baseline,43787.301848
2,1-5_months,female,2022,232,baseline,43689.518138
3,1-5_months,female,2022,357,baseline,43650.172485
...,...,...,...,...,...,...
716,late_neonatal,male,2026,650,baseline,6258.131417
717,late_neonatal,male,2026,680,baseline,6279.897331
718,late_neonatal,male,2026,829,baseline,6308.962355
719,late_neonatal,male,2026,946,baseline,6238.913073


In [106]:
vo.stratify(data.wasting_state_person_time, ['cause'])

,cause,input_draw,scenario,value
0,mild_child_wasting,29,baseline,1.095975e+06
1,mild_child_wasting,223,baseline,1.098109e+06
2,mild_child_wasting,232,baseline,1.098434e+06
3,mild_child_wasting,357,baseline,1.101842e+06
...,...,...,...,...
44,susceptible_to_child_wasting,650,baseline,3.565565e+06
45,susceptible_to_child_wasting,680,baseline,3.578780e+06
46,susceptible_to_child_wasting,829,baseline,3.583406e+06
47,susceptible_to_child_wasting,946,baseline,3.557827e+06


In [108]:
'|'.join(data.wasting_transition_count.measure.unique())

'mild_child_wasting_to_moderate_acute_malnutrition_event_count|mild_child_wasting_to_susceptible_to_child_wasting_event_count|moderate_acute_malnutrition_to_mild_child_wasting_event_count|moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count|severe_acute_malnutrition_to_mild_child_wasting_event_count|severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count|susceptible_to_child_wasting_to_mild_child_wasting_event_count'

In [109]:
'|'.join(data.wasting_state_person_time.measure.unique())

'state_person_time'

In [111]:
'|'.join([str(i) for i in [1,2,3]])

'1|2|3'

In [117]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    numerator_broadcast='measure',
    denominator_broadcast='cause'
)

,year,age,sex,input_draw,scenario,measure,cause,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,mild_child_wasting,3.268528,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
1,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,moderate_acute_malnutrition,8.254240,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
2,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,severe_acute_malnutrition,31.588639,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
3,2022,1-5_months,female,29,baseline,mild_child_wasting_to_moderate_acute_malnutrit...,susceptible_to_child_wasting,0.948723,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...,...
20156,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,mild_child_wasting,0.306930,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
20157,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,moderate_acute_malnutrition,0.673521,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
20158,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,severe_acute_malnutrition,1.797606,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
20159,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting_to_mild_child_was...,susceptible_to_child_wasting,0.095641,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1


In [118]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    denominator_broadcast='cause',
)

,year,age,sex,input_draw,scenario,cause,value,numerator_measure,denominator_measure,multiplier
0,2022,1-5_months,female,29,baseline,mild_child_wasting,8.289502,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
1,2022,1-5_months,female,29,baseline,moderate_acute_malnutrition,20.934053,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
2,2022,1-5_months,female,29,baseline,severe_acute_malnutrition,80.113760,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
3,2022,1-5_months,female,29,baseline,susceptible_to_child_wasting,2.406110,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
...,...,...,...,...,...,...,...,...,...,...
2876,2026,late_neonatal,male,946,baseline,mild_child_wasting,10.253494,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
2877,2026,late_neonatal,male,946,baseline,moderate_acute_malnutrition,22.500028,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
2878,2026,late_neonatal,male,946,baseline,severe_acute_malnutrition,60.051854,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1
2879,2026,late_neonatal,male,946,baseline,susceptible_to_child_wasting,3.195047,mild_child_wasting_to_moderate_acute_malnutrit...,state_person_time,1


In [119]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    denominator_broadcast='cause',
    reset_index=False
)

value
year age           sex    input_draw scenario cause                                  
2022 1-5_months    female 29         baseline mild_child_wasting             8.289502
                                              moderate_acute_malnutrition   20.934053
                                              severe_acute_malnutrition     80.113760
                                              susceptible_to_child_wasting   2.406110
...                                                                               ...
2026 late_neonatal male   946        baseline mild_child_wasting            10.253494
                                              moderate_acute_malnutrition   22.500028
                                              severe_acute_malnutrition     60.051854
                                              susceptible_to_child_wasting   3.195047

[2880 rows x 1 columns]

In [120]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata=['year', 'age', 'sex'],
    denominator_broadcast='cause',
    reset_index=False,
    record_inputs=True
)

value  \
year age           sex    input_draw scenario cause                                     
2022 1-5_months    female 29         baseline mild_child_wasting             8.289502   
                                              moderate_acute_malnutrition   20.934053   
                                              severe_acute_malnutrition     80.113760   
                                              susceptible_to_child_wasting   2.406110   
...                                                                               ...   
2026 late_neonatal male   946        baseline mild_child_wasting            10.253494   
                                              moderate_acute_malnutrition   22.500028   
                                              severe_acute_malnutrition     60.051854   
                                              susceptible_to_child_wasting   3.195047   

                                                                                                            numerator_measure  \
year age           sex    input_draw scenario cause                                                                             
2022 1-5_months    female 29         baseline mild_child_wasting            mild_child_wasting_to_moderate_acute_malnutrit...   
                                              moderate_acute_malnutrition   mild_child_wasting_to_moderate_acute_malnutrit...   
                                              severe_acute_malnutrition     mild_child_wasting_to_moderate_acute_malnutrit...   
                                              susceptible_to_child_wasting  mild_child_wasting_to_moderate_acute_malnutrit...   
...                                                                                                                       ...   
2026 late_neonatal male   946        baseline mild_child_wasting            mild_child_wasting_to_moderate_acute_malnutrit...   
                                              moderate_acute_malnutrition   mild_child_wasting_to_moderate_acute_malnutrit...   
                                              severe_acute_malnutrition     mild_child_wasting_to_moderate_acute_malnutrit...   
                                              susceptible_to_child_wasting  mild_child_wasting_to_moderate_acute_malnutrit...   

                                                                           denominator_measure  \
year age           sex    input_draw scenario cause                                              
2022 1-5_months    female 29         baseline mild_child_wasting             state_person_time   
                                              moderate_acute_malnutrition    state_person_time   
                                              severe_acute_malnutrition      state_person_time   
                                              susceptible_to_child_wasting   state_person_time   
...                                                                                        ...   
2026 late_neonatal male   946        baseline mild_child_wasting             state_person_time   
                                              moderate_acute_malnutrition    state_person_time   
                                              severe_acute_malnutrition      state_person_time   
                                              susceptible_to_child_wasting   state_person_time   

                                                                            multiplier  
year age           sex    input_draw scenario cause                                     
2022 1-5_months    female 29         baseline mild_child_wasting                     1  
                                              moderate_acute_malnutrition            1  
                                              severe_acute_malnutrition              1  
                                              susceptible_to_child_wasting           1  
...                                                                

In [130]:
vo.ratio(
    data.wasting_transition_count,
    data.wasting_state_person_time,
    strata='year',
    denominator_broadcast='cause',
    reset_index=False,
    record_inputs=True
)

value  \
year input_draw scenario cause                                     
2022 29         baseline mild_child_wasting             7.455185   
                         moderate_acute_malnutrition   20.587418   
                         severe_acute_malnutrition     86.748025   
                         susceptible_to_child_wasting   2.227517   
...                                                          ...   
2026 946        baseline mild_child_wasting             7.568245   
                         moderate_acute_malnutrition   20.714230   
                         severe_acute_malnutrition     84.585419   
                         susceptible_to_child_wasting   2.356075   

                                                                                       numerator_measure  \
year input_draw scenario cause                                                                             
2022 29         baseline mild_child_wasting            mild_child_wasting_to_moderate_acute_malnutrit...   
                         moderate_acute_malnutrition   mild_child_wasting_to_moderate_acute_malnutrit...   
                         severe_acute_malnutrition     mild_child_wasting_to_moderate_acute_malnutrit...   
                         susceptible_to_child_wasting  mild_child_wasting_to_moderate_acute_malnutrit...   
...                                                                                                  ...   
2026 946        baseline mild_child_wasting            mild_child_wasting_to_moderate_acute_malnutrit...   
                         moderate_acute_malnutrition   mild_child_wasting_to_moderate_acute_malnutrit...   
                         severe_acute_malnutrition     mild_child_wasting_to_moderate_acute_malnutrit...   
                         susceptible_to_child_wasting  mild_child_wasting_to_moderate_acute_malnutrit...   

                                                      denominator_measure  \
year input_draw scenario cause                                              
2022 29         baseline mild_child_wasting             state_person_time   
                         moderate_acute_malnutrition    state_person_time   
                         severe_acute_malnutrition      state_person_time   
                         susceptible_to_child_wasting   state_person_time   
...                                                                   ...   
2026 946        baseline mild_child_wasting             state_person_time   
                         moderate_acute_malnutrition    state_person_time   
                         severe_acute_malnutrition      state_person_time   
                         susceptible_to_child_wasting   state_person_time   

                                                       multiplier  
year input_draw scenario cause                                     
2022 29         baseline mild_child_wasting                     1  
                         moderate_acute_malnutrition            1  
                         severe_acute_malnutrition              1  
                         susceptible_to_child_wasting           1  
...                                                           ...  
2026 946        baseline mild_child_wasting                     1  
                         moderate_acute_malnutrition            1  
                         severe_acute_malnutrition              1  
                         susceptible_to_child_wasting           1  

[240 rows x 4 columns]

In [121]:
data.population

,measure,input_draw,scenario,value
0,total_population,29,baseline,2089439.0
1,total_population,223,baseline,2090788.0
2,total_population,232,baseline,2090287.0
3,total_population,357,baseline,2088756.0
...,...,...,...,...
56,total_population_untracked,650,baseline,988204.0
57,total_population_untracked,680,baseline,990901.0
58,total_population_untracked,829,baseline,991915.0
59,total_population_untracked,946,baseline,988882.0


In [123]:
data.population.measure.unique()

array(['total_population', 'total_population_dead',
       'total_population_living', 'total_population_tracked',
       'total_population_untracked'], dtype=object)

In [124]:
data.population.input_draw.nunique()

12

In [122]:
data.deaths

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,32.0,cat4,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat3,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat2,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,3.0,cat4,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,cat3,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,1.0,cat2,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,1.0,cat1,2_to_4


In [136]:
# 17280 = 4 stunting states x 12 draws x 5 years x 2 sexes x 6 ages x 6 causes
17280 / 4 / 12 / 5 / 2 / 6 / 6

1.0

In [133]:
data.deaths.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'other_causes',
       'severe_acute_malnutrition'], dtype=object)

In [135]:
data.deaths.age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [140]:
def describe(data, **describe_kwargs):
    if 'percentiles' not in describe_kwargs:
        describe_kwargs['percentiles'] = [.025, .975]
    groupby_cols = data.columns.difference([vo.DRAW_COLUMN, vo.VALUE_COLUMN]).to_list()
    return data.groupby(groupby_cols)[vo.VALUE_COLUMN].describe(**describe_kwargs)

describe(data.wasting_state_person_time)

count  \
age           cause                        measure           scenario sex    year          
1-5_months    mild_child_wasting           state_person_time baseline female 2022   12.0   
                                                                             2023   12.0   
                                                                             2024   12.0   
                                                                             2025   12.0   
...                                                                                  ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023   12.0   
                                                                             2024   12.0   
                                                                             2025   12.0   
                                                                             2026   12.0   

                                                                                          mean  \
age           cause                        measure           scenario sex    year                
1-5_months    mild_child_wasting           state_person_time baseline female 2022  8826.488935   
                                                                             2023  8794.313826   
                                                                             2024  8836.645562   
                                                                             2025  8808.098677   
...                                                                                        ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023  4174.045403   
                                                                             2024  4199.450605   
                                                                             2025  4182.792494   
                                                                             2026  4172.952202   

                                                                                          std  \
age           cause                        measure           scenario sex    year               
1-5_months    mild_child_wasting           state_person_time baseline female 2022   88.753512   
                                                                             2023  165.691577   
                                                                             2024  137.494408   
                                                                             2025  141.141967   
...                                                                                       ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023   30.327060   
                                                                             2024   25.089339   
                                                                             2025   40.514449   
                                                                             2026   24.003145   

                                                                                           min  \
age           cause                        measure           scenario sex    year                
1-5_months    mild_child_wasting           state_person_time baseline female 2022  8700.517454   
                                                                             2023  8607.275838   
                                                                             2024  8702.147844   
                                                                             2025  8655.026694   
...                                                                                        ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023  4101.519507   
                                                                             2024  4152.565366   
                                         

In [142]:
df = describe(data.wasting_state_person_time)
df

count  \
age           cause                        measure           scenario sex    year          
1-5_months    mild_child_wasting           state_person_time baseline female 2022   12.0   
                                                                             2023   12.0   
                                                                             2024   12.0   
                                                                             2025   12.0   
...                                                                                  ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023   12.0   
                                                                             2024   12.0   
                                                                             2025   12.0   
                                                                             2026   12.0   

                                                                                          mean  \
age           cause                        measure           scenario sex    year                
1-5_months    mild_child_wasting           state_person_time baseline female 2022  8826.488935   
                                                                             2023  8794.313826   
                                                                             2024  8836.645562   
                                                                             2025  8808.098677   
...                                                                                        ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023  4174.045403   
                                                                             2024  4199.450605   
                                                                             2025  4182.792494   
                                                                             2026  4172.952202   

                                                                                          std  \
age           cause                        measure           scenario sex    year               
1-5_months    mild_child_wasting           state_person_time baseline female 2022   88.753512   
                                                                             2023  165.691577   
                                                                             2024  137.494408   
                                                                             2025  141.141967   
...                                                                                       ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023   30.327060   
                                                                             2024   25.089339   
                                                                             2025   40.514449   
                                                                             2026   24.003145   

                                                                                           min  \
age           cause                        measure           scenario sex    year                
1-5_months    mild_child_wasting           state_person_time baseline female 2022  8700.517454   
                                                                             2023  8607.275838   
                                                                             2024  8702.147844   
                                                                             2025  8655.026694   
...                                                                                        ...   
late_neonatal susceptible_to_child_wasting state_person_time baseline male   2023  4101.519507   
                                                                             2024  4152.565366   
                                         

In [144]:
df['std'] * (df['count'] / (df['count']-1)) / np.sqrt(df['count'])

age            cause                         measure            scenario  sex     year
1-5_months     mild_child_wasting            state_person_time  baseline  female  2022    27.950108
                                                                                  2023    52.179314
                                                                                  2024    43.299509
                                                                                  2025    44.448192
                                                                                            ...    
late_neonatal  susceptible_to_child_wasting  state_person_time  baseline  male    2023     9.550547
                                                                                  2024     7.901093
                                                                                  2025    12.758743
                                                                                  2026     7.559030
Length: 240, 